In [2]:

import numpy as np
import pandas as pd



In [5]:
import os as os


os. getcwd() 
os.chdir(r"C:\Users\hanne\Documents\2019-2020\ESCP Courses\Python") #Had to put r in front as otherwise it wouldn't see the string as a raw string 


In [7]:

dataTr = pd.read_csv("train.csv")
datats = pd.read_csv("test1.csv")

In [8]:
dataTr
original_headers = list(dataTr.columns.values)
original_headers

['label',
 'id',
 'visitTime',
 'purchaseTime',
 'hour',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'N1',
 'C9',
 'N2',
 'N3',
 'N4',
 'N5',
 'N6',
 'N7',
 'N8',
 'N9',
 'N10',
 'C10',
 'C11',
 'C12']

In [9]:

## Replacing class -1 with 0, because easier to work with
dataTr["label"].replace({-1: 0}, inplace=True)
datats["label"].replace({-1: 0}, inplace=True)

In [11]:

#Checking Data
for i in dataTr.columns:
    print ("---- %s ---" % i)
    print (dataTr[i].value_counts())

---- label ---
0    31371
1       57
Name: label, dtype: int64
---- id ---
399357    1
159399    1
525994    1
278189    1
448176    1
         ..
68628     1
590869    1
476883    1
5143      1
327680    1
Name: id, Length: 31428, dtype: int64
---- visitTime ---
19328699    6
19319226    6
19325796    5
19314062    5
19310993    5
           ..
19310525    1
19283902    1
19314623    1
19318721    1
19300352    1
Name: visitTime, Length: 24587, dtype: int64
---- purchaseTime ---
-1           31371
 19307708        1
 19319002        1
 19306613        1
 19298361        1
 19330955        1
 19318637        1
 19324431        1
 19295049        1
 19331883        1
 19325720        1
 19344902        1
 19331996        1
 19325359        1
 19323160        1
 19327407        1
 19327835        1
 19323177        1
 19300350        1
 19344084        1
 19345863        1
 19312951        1
 19304723        1
 19335360        1
 19337379        1
 19302544        1
 19325069        1
 1

In [12]:

## Checking, if there are missing values
dataTr.isnull().sum()

label           0
id              0
visitTime       0
purchaseTime    0
hour            0
C1              0
C2              0
C3              0
C4              0
C5              0
C6              0
C7              0
C8              0
N1              0
C9              0
N2              0
N3              0
N4              0
N5              0
N6              0
N7              0
N8              0
N9              0
N10             0
C10             0
C11             0
C12             0
dtype: int64

In [15]:


## Split train - data into X and Y variable
X = dataTr.drop(["label","visitTime","hour","purchaseTime"], axis=1)
Y = dataTr["label"]

In [16]:

## Modeling with Logistic Regression
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.33, stratify=Y)


In [17]:

## Oversampling the data set, as one class is underrepresented
import collections
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
print('Original dataset shape %s' % Counter(Y_train))
sm = SMOTE(random_state=42, sampling_strategy=0.3)
X_res, Y_res = sm.fit_resample(X_train,Y_train)
rs = RandomUnderSampler(random_state =42, sampling_strategy=0.7)
X_res, Y_res = rs.fit_resample(X_res, Y_res)
print('Resample dataset shape %s' % Counter(Y_res))


Original dataset shape Counter({0: 21018, 1: 38})
Resample dataset shape Counter({0: 9007, 1: 6305})


Using TensorFlow backend.


In [18]:

## Run Logistic Regression Model
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_res, Y_res)
predictions = logmodel.predict(X_test)


In [19]:


## Classification Report
from sklearn.metrics import classification_report
print(classification_report(Y_test, predictions))


              precision    recall  f1-score   support

           0       1.00      0.83      0.90     10353
           1       0.01      0.58      0.01        19

    accuracy                           0.82     10372
   macro avg       0.50      0.70      0.46     10372
weighted avg       1.00      0.82      0.90     10372



In [20]:

## Create confusion matrix for better understanding
from sklearn.metrics import confusion_matrix
confusion_matrix(predictions,Y_test)

array([[8544,    8],
       [1809,   11]], dtype=int64)

In [21]:

## Random Forest Model - Check with model before
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 500, max_depth = 3, random_state = 42)
rf.fit(X_res, Y_res)
predictionsrf = rf.predict(X_test)

In [22]:
print(classification_report(Y_test, predictionsrf))


              precision    recall  f1-score   support

           0       1.00      0.99      1.00     10353
           1       0.17      0.95      0.29        19

    accuracy                           0.99     10372
   macro avg       0.58      0.97      0.64     10372
weighted avg       1.00      0.99      0.99     10372



In [23]:
confusion_matrix(predictionsrf,Y_test)


array([[10264,     1],
       [   89,    18]], dtype=int64)

In [24]:
## Display ROC curves for both models
import matplotlib.pyplot as plt
from sklearn.metrics import plot_roc_curve
ax = plt.gca()
lr_curve = plot_roc_curve(logmodel, X_test, Y_test,ax=ax, alpha=0.8)
rf_curve = plot_roc_curve(rf, X_test, Y_test,ax=ax, alpha=0.8)
plt.show()


<Figure size 640x480 with 1 Axes>

In [25]:
#Randomforest 
## Fit the model to test dataset
datats

,label,id,visitTime,purchaseTime,hour,C1,C2,C3,C4,C5,...,N4,N5,N6,N7,N8,N9,N10,C10,C11,C12
0,0,1,19360733,-1,0,2569487005,403381353,681762206,652481754,2967512603,...,0,0,0,0,0,0,0,2128770044,3617851689,4220791646
1,0,2,19360734,-1,0,2416811818,1235983246,3653557526,652481754,2116767313,...,0,0,0,0,0,0,0,4256459078,2379633508,1284343215
2,0,3,19360735,-1,0,3765056181,1235983246,1310333812,652481754,2116767313,...,0,0,0,0,0,0,0,2840358457,3831440054,1213938795
3,0,4,19360737,-1,0,955802445,403381353,2290282995,652481754,2967512603,...,0,0,0,0,0,0,0,4039873369,535829057,3828748840
4,0,5,19360752,-1,0,2569487005,2517724481,2463986301,2308170622,2967512603,...,0,0,0,0,0,0,0,907610626,3617851689,4220791646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32081,0,32082,19446873,-1,23,2569487005,2018710401,887116886,652481754,2967512603,...,0,0,0,0,0,0,0,1546561973,3617851689,4220791646
32082,0,32083,19446914,-1,23,2569487005,403381353,3475347077,652481754,2967512603,...,0,0,0,0,0,0,0,3227742820,3617851689,4220791646
32083,0,32084,19446925,-1,23,2569487005,403381353,3475347077,652481754,2967512603,...,0,0,0,0,0,0,0,4069315994,3617851689,4220791646
32084,0,32085,19446931,-1,23,2569487005,2541203883,448895441,2308170622,2967512603,...,0,0,0,0,0,0,0,2724771230,3617851689,4220791646


In [26]:

## Split test - data into X and Y variable
Xts = datats.drop(["label","visitTime","hour","purchaseTime"], axis=1)
Yts = datats["label"]

In [27]:

## Predict probabilities based on the test data set
predictionsts = rf.predict_proba(Xts)
predictionsts

array([[0.93276602, 0.06723398],
       [0.957511  , 0.042489  ],
       [0.94333475, 0.05666525],
       ...,
       [0.9072219 , 0.0927781 ],
       [0.83417299, 0.16582701],
       [0.82132042, 0.17867958]])

In [28]:

## Create data frame out of predictions and rename columns
prediction_frame = pd.DataFrame(predictionsts)
nf = prediction_frame.rename(columns = {0:"No_Purchase",1:"Purchase"})
nf

,No_Purchase,Purchase
0,0.932766,0.067234
1,0.957511,0.042489
2,0.943335,0.056665
3,0.920951,0.079049
4,0.938220,0.061780
...,...,...
32081,0.836621,0.163379
32082,0.903729,0.096271
32083,0.907222,0.092778
32084,0.834173,0.165827


In [29]:

## Merge two datasets and only keep relevant columns
result = pd.concat([Xts, nf], axis=1, join='inner')
deliverable = result[["id","Purchase"]]
deliverable

,id,Purchase
0,1,0.067234
1,2,0.042489
2,3,0.056665
3,4,0.079049
4,5,0.061780
...,...,...
32081,32082,0.163379
32082,32083,0.096271
32083,32084,0.092778
32084,32085,0.165827


In [30]:
deliverable.to_csv("Hannes_Purchase_Probability.csv")
